**Использование псевдоразметки. ДЗ.**

In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import random
import numpy as np

In [2]:
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)
torch.backends.cudnn.deterministic = True

Начнем с загрузки датасета. Речевые данные (и модели, обучаемые на них) очень тяжелые, поэтому мы обойдемся чем-нибудь попроще.

In [3]:
train_dataset = \
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
test_dataset = \
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
len(train_dataset), len(test_dataset)

(60000, 10000)

Итак, трейн состоит из 60000 картинок цифр. Для того, чтобы получше увидеть эффект от псевдолейблов, мы оставим только 100 этих картинок в качестве размеченных данных. Остальные 59900 будут в качестве неразмеченных. 

На масштабах 100 записей могут проявиться неприятные эффекты, если какие-то из классов не будут достаточно хорошо представлены. Чтобы этого избежать, будем аккуратно семплировать. Самый простой вариант - просто случайно разделять, пока не получится удачное разбиение.

Для начала определим удачность разбиения. Будем считать размеченный датасет хорошим, если из 100 примеров в нем есть хотя бы по 8 представителей каждого класса. Напишите функцию, которая делает такую проверку.

In [5]:
def check_dataset(dataset, labels_num=10, min_samples_num=8):
  labels = np.array([item[1] for item in dataset])
  unique_labels, counts = np.unique(labels, return_counts=True)
  return len(unique_labels) == labels_num and np.all(counts >= min_samples_num)

In [6]:
sampling_iteration = 0
while True:
    labeled_train_dataset, unlabeled_train_dataset = torch.utils.data.random_split(train_dataset, [100, 59900])
    if check_dataset(labeled_train_dataset):
        break
    sampling_iteration += 1
print(f'Split the dataset after {sampling_iteration} resamplings')

Split the dataset after 32 resamplings


In [7]:
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False)
labeled_train_loader = torch.utils.data.DataLoader(
    labeled_train_dataset, batch_size=64, shuffle=True)
unlabeled_train_loader = torch.utils.data.DataLoader(
    unlabeled_train_dataset, batch_size=64, shuffle=False)

Теперь, когда мы получили данные, определим архитектуру сети. Возьмем простую сверточную сетку с droupout'ом.

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 40, kernel_size=5)
        self.dropout = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(640, 150)
        self.fc2 = nn.Linear(150, 10)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = x.view(-1, 640)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.log_softmax(x)
        return x

Опишем вспомогательные функции.

In [9]:
def train(epoch_idx, model, optimizer, train_loader, loss_func=F.nll_loss):
    model.train()
    for batch_idx, (x, target) in enumerate(train_loader):
        x, target = x.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()

In [10]:
def test(epoch_idx, model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for x, target in test_loader:
            x, target = x.cuda(), target.cuda()
            output = model(x)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('Epoch {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        epoch_idx, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
def predict(model, loader):
    model.eval()
    result = []
    with torch.no_grad():
        for x, _ in loader:
            result.append(model(x.cuda()))
    return torch.cat(result)

Создадим модель и обучим ее на нашем размеченном датасете.

In [12]:
model = Net().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [13]:
for i in range(400):
    train(i, model, optimizer, labeled_train_loader)
    if i % 10 == 0:
        test(i, model, test_loader)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/usr/local/lib/python3.8/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch 0: Average loss: 2.2969, Accuracy: 985/10000 (10%)
Epoch 10: Average loss: 1.7321, Accuracy: 5573/10000 (56%)
Epoch 20: Average loss: 0.7752, Accuracy: 7474/10000 (75%)
Epoch 30: Average loss: 0.5837, Accuracy: 8025/10000 (80%)
Epoch 40: Average loss: 0.7819, Accuracy: 7384/10000 (74%)
Epoch 50: Average loss: 0.5545, Accuracy: 8304/10000 (83%)
Epoch 60: Average loss: 0.5442, Accuracy: 8506/10000 (85%)
Epoch 70: Average loss: 0.5475, Accuracy: 8538/10000 (85%)
Epoch 80: Average loss: 0.5823, Accuracy: 8570/10000 (86%)
Epoch 90: Average loss: 0.6238, Accuracy: 8607/10000 (86%)
Epoch 100: Average loss: 0.6114, Accuracy: 8562/10000 (86%)
Epoch 110: Average loss: 0.6757, Accuracy: 8571/10000 (86%)
Epoch 120: Average loss: 0.6722, Accuracy: 8563/10000 (86%)
Epoch 130: Average loss: 0.6936, Accuracy: 8601/10000 (86%)
Epoch 140: Average loss: 0.7402, Accuracy: 8525/10000 (85%)
Epoch 150: Average loss: 0.6707, Accuracy: 8667/10000 (87%)
Epoch 160: Average loss: 0.6714, Accuracy: 8514/1000

Теперь попробуем побить этот результат с помощью псевдолейблов. Напишем функцию, которая принимает модель и возращает DataLoader с хард-лейблами, и запустим обучение.

In [14]:
def get_pseudo_loader(model):
    dataset_samples = [item[0] for item in list(unlabeled_train_dataset)]
    dataset_labels = []
    loader = torch.utils.data.DataLoader(
        unlabeled_train_dataset,
        batch_size=1024
    )
    logits = predict(model, loader)
    dataset_labels = torch.max(logits, 1)[1]
    
    dataset = list(zip(dataset_samples, dataset_labels))

    return torch.utils.data.DataLoader(
        dataset, batch_size=64, shuffle=True
        )


In [15]:
model_hard = Net().cuda()
model_hard.load_state_dict(model.state_dict())
optimizer_hard = torch.optim.SGD(model_hard.parameters(), lr=0.1)

In [16]:
hard_labeled_loader = get_pseudo_loader(model)
for i in range(10):
    train(i, model_hard, optimizer_hard, hard_labeled_loader)
    train(i, model_hard, optimizer_hard, labeled_train_loader)
    test(i, model_hard, test_loader)

Epoch 0: Average loss: 0.5234, Accuracy: 8676/10000 (87%)
Epoch 1: Average loss: 0.6100, Accuracy: 8580/10000 (86%)
Epoch 2: Average loss: 0.6465, Accuracy: 8602/10000 (86%)
Epoch 3: Average loss: 0.6380, Accuracy: 8528/10000 (85%)
Epoch 4: Average loss: 0.5533, Accuracy: 8682/10000 (87%)
Epoch 5: Average loss: 0.6062, Accuracy: 8583/10000 (86%)
Epoch 6: Average loss: 0.6020, Accuracy: 8614/10000 (86%)
Epoch 7: Average loss: 0.6768, Accuracy: 8606/10000 (86%)
Epoch 8: Average loss: 0.6546, Accuracy: 8628/10000 (86%)
Epoch 9: Average loss: 0.6762, Accuracy: 8631/10000 (86%)


**Итеративная псевдоразметка.**

Мы уже видим небольшое улучшение, но можно пойти дальше.

In [17]:
model_hard_iter = Net().cuda()
model_hard_iter.load_state_dict(model.state_dict())
optimizer_hard_iter = torch.optim.SGD(model_hard_iter.parameters(), lr=0.1)

In [18]:
for i in range(20):
    hard_labeled_loader = get_pseudo_loader(model_hard_iter)
    train(i, model_hard_iter, optimizer_hard_iter, hard_labeled_loader)
    train(i, model_hard_iter, optimizer_hard_iter, labeled_train_loader)
    test(i, model_hard_iter, test_loader)

Epoch 0: Average loss: 0.5203, Accuracy: 8725/10000 (87%)
Epoch 1: Average loss: 0.4623, Accuracy: 8877/10000 (89%)
Epoch 2: Average loss: 0.5010, Accuracy: 8867/10000 (89%)
Epoch 3: Average loss: 0.4642, Accuracy: 9027/10000 (90%)
Epoch 4: Average loss: 0.4574, Accuracy: 9061/10000 (91%)
Epoch 5: Average loss: 0.3897, Accuracy: 9158/10000 (92%)
Epoch 6: Average loss: 0.4120, Accuracy: 9210/10000 (92%)


KeyboardInterrupt: ignored

**Оценивание.**

В предыдущем пункте нужно получить accuracy 91% или выше (5 баллов).

Следующие шаги:

Модифицировать функцию `get_pseudo_loader`, чтобы она могла возвращать софт-лейблы (+1 балл).

Правильно запустить обучение - в качестве лосса используем KL-дивергенцию. Получить accuracy 90% или выше. (+3 балла).

Интуитивно кажется, что модель не должна ничему учиться, т.к. ее выход будет полностью совпадать с софт-лейблами. Напишите (текстом), почему тем не менее удается сильно выиграть относительно бейзлайна. (+1 балл).

In [19]:
model_soft_iter = Net().cuda()
model_soft_iter.load_state_dict(model.state_dict())
optimizer_soft_iter = torch.optim.SGD(model_soft_iter.parameters(), lr=0.1)

In [20]:
def get_pseudo_loader(model, soft=False):
    dataset_samples = [item[0] for item in list(unlabeled_train_dataset)]
    dataset_labels = []
    loader = torch.utils.data.DataLoader(
        unlabeled_train_dataset,
        batch_size=1024
    )
    logits = predict(model, loader)
    if soft:
      dataset_labels = logits
    else:
      dataset_labels = torch.max(logits, 1)[1]
    
    dataset = list(zip(dataset_samples, dataset_labels))

    return torch.utils.data.DataLoader(
        dataset, batch_size=64, shuffle=True
        )

In [21]:
def train_with_kl(epoch_idx, model, optimizer, train_loader):
    model.train()
    loss_func = torch.nn.KLDivLoss(reduction='batchmean', log_target=True)
    for batch_idx, (x, target) in enumerate(train_loader):
        x, target = x.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(x)
        pred = F.log_softmax(output, dim=1)
        target = F.log_softmax(target, dim=1)
        loss = loss_func(pred, target)
        loss.backward()
        optimizer.step()

In [23]:
for i in range(20):
    soft_labeled_loader = get_pseudo_loader(model_soft_iter, soft=True)
    train_with_kl(i, model_soft_iter, optimizer_soft_iter, soft_labeled_loader)
    train(i, model_soft_iter, optimizer_soft_iter, labeled_train_loader)
    test(i, model_soft_iter, test_loader)

Epoch 0: Average loss: 0.2714, Accuracy: 9272/10000 (93%)
Epoch 1: Average loss: 0.2691, Accuracy: 9287/10000 (93%)
Epoch 2: Average loss: 0.2649, Accuracy: 9307/10000 (93%)
Epoch 3: Average loss: 0.2811, Accuracy: 9205/10000 (92%)
Epoch 4: Average loss: 0.2694, Accuracy: 9317/10000 (93%)
Epoch 5: Average loss: 0.2673, Accuracy: 9329/10000 (93%)
Epoch 6: Average loss: 0.2637, Accuracy: 9265/10000 (93%)
Epoch 7: Average loss: 0.2669, Accuracy: 9245/10000 (92%)
Epoch 8: Average loss: 0.2641, Accuracy: 9237/10000 (92%)
Epoch 9: Average loss: 0.2688, Accuracy: 9233/10000 (92%)
Epoch 10: Average loss: 0.2633, Accuracy: 9247/10000 (92%)
Epoch 11: Average loss: 0.2623, Accuracy: 9275/10000 (93%)
Epoch 12: Average loss: 0.2669, Accuracy: 9206/10000 (92%)
Epoch 13: Average loss: 0.2627, Accuracy: 9211/10000 (92%)
Epoch 14: Average loss: 0.2637, Accuracy: 9216/10000 (92%)
Epoch 15: Average loss: 0.2605, Accuracy: 9238/10000 (92%)


KeyboardInterrupt: ignored